In [321]:
from tkinter import *
import tkinter as tk
from tkinter.filedialog import askopenfilename
from pm4py.visualization.dfg import visualizer as dfg_visualizer
import os
from PIL import Image, ImageTk
import pm4py
import pandas as pd


In [322]:
# global caminho_do_arquivo

### função que escolhe o caminho 

In [323]:
def escolhe_arquivo():
    caminho_do_arquivo = askopenfilename(filetypes=[("Arquivo CSV", "*.csv"), ("Arquivo Excel", "*.xlsx"), ('só para testes PNG', '*.png'), ('xes', '*.xes')])
    if caminho_do_arquivo:
        
        label_arquivo.config(text=f"Arquivo selecionado: {caminho_do_arquivo}") # Calma é pra não estar definido msm

        #Mudar a chama dessa função quando for criar a função que trata os dados
        cria_grafo_dfg(caminho_do_arquivo)
        

### função cria o gráfico

In [324]:
def cria_grafo_dfg(caminho):
    #Checando se o arquivo é xlsx ou csv
    tipoArq = caminho.split('.')
    
    if tipoArq[-1] in 'csv':
        log = pm4py.format_dataframe(pd.read_csv(caminho), case_id='ID', activity_key='Ocorrência', timestamp_key='Início')

    elif tipoArq[-1] in 'xlsx':
        log = pm4py.format_dataframe(pd.read_excel(caminho), case_id='ID', activity_key='Ocorrência', timestamp_key='Início') # depois arrumar um jeito de o usuario selecionar as colunas case_id activity_key e timestamp_key

    elif tipoArq[-1] in 'xes':
        log = pm4py.format_dataframe(pm4py.read_xes(caminho), case_id='ID', activity_key='Ocorrência', timestamp_key='Início') # depois arrumar um jeito de o usuario selecionar as colunas case_id 

    ##----------------------------------------------------------------------------------------------------------------------------------------
    log['ID'] = log['ID'].astype(str)

    dfg, start_activities, end_activities = pm4py.discover_dfg(log)
    grafo = dfg_visualizer.apply(dfg, log=log)
    dfg_visualizer.save(grafo, "dfg_grafico.png")
    
    ##----------------------------------------------------------------------------------------------------------------------------------------
    lable_auxImagem.config(text="Imagem do DFG salva como 'dfg_grafico.png'")
    print("Imagem do DFG salva como 'dfg_grafico.png'")

    

    exibe_grafico("dfg_grafico.png")

### Função que vai plotar os gráficos

In [325]:
def exibe_grafico(caminho):
    
    imagem_grafico = PhotoImage(file=caminho, master=frame_grafico)
    imagem_grafico = imagem_grafico.subsample(1,1)
    fig_grafico = Label(frame_grafico, image=imagem_grafico)
    fig_grafico.image = imagem_grafico
    fig_grafico.pack()

### INSTANCIANDO A JANELA

In [326]:
raiz = Tk(className='Primeira janela de teste para a interface')
raiz.geometry('960x540')

''

### Criando o frame de seleção de arquivos

In [327]:
frame1 = Frame(raiz) #bg="blue"
frame1.pack(side='top',fill=tk.BOTH)

### Botão de seleção de arquivos ###

In [328]:
arq = PhotoImage(file='iconArq.png', master=frame1)
arq = arq.subsample(2,2)
fig1 = Label(frame1, image=arq)
fig1.image = arq

botao = Button(frame1, image=arq, command=escolhe_arquivo, bd=0, highlightthickness=0, bg= 'green')
botao.pack(side='left')

label_arquivo = Label(frame1, text="Nenhum arquivo selecionado")
label_arquivo.pack(pady=10)

lable_auxImagem= Label(frame1, text="Nenhma Imagem Gerada")
lable_auxImagem.pack(pady=10)

### Botões de seleção de Gráfico (TESTE) ###

In [329]:

# botaoFrequencia = Button(frame1, text='Grafico Frêquencia', command=cria_grafo_dfg, bd=5, relief="ridge", bg= 'skyblue')
# botaoFrequencia.pack(side='right', pady=10)

# botaoPetriNet = Button(frame1, text='Grafico Rede de Petri', command=escolhe_arquivo, bd=5, relief="ridge", bg= 'skyblue')
# botaoPetriNet.pack(side='right', pady=10)

# botaoPerformance = Button(frame1, text='Grafico Performance', command=escolhe_arquivo, bd=5, relief="ridge", bg= 'skyblue')
# botaoPerformance.pack(side='right', pady=10)


### Criando o frame Lateral

In [330]:
frame_lateral = Frame(raiz)
frame_lateral.pack(side = 'right',pady=20, fill=tk.BOTH)

# Criando A lable na lateral direita
label_direita_sup = Label(frame_lateral, padx=40, pady=10, borderwidth=2, relief="solid")
label_direita_sup.pack(fill=tk.Y, expand=True)

# Criando um frame para cada conjunto de label e escala
frame_escala1 = tk.Frame(label_direita_sup)
frame_escala1.pack(side='left', padx=5, pady=10)

frame_escala2 = tk.Frame(label_direita_sup)
frame_escala2.pack(side='left', padx=5, pady=10)

# Adicionando rótulo e escala no primeiro frame
label_escala1 = tk.Label(frame_escala1, text="Activities")
label_escala1.pack()

escala1 = Scale(frame_escala1, from_=100, to=0, orient=tk.VERTICAL)
escala1.set(0)
escala1.pack(side = 'left')

# Adicionando rótulo e escala no segundo frame
label_escala2 = tk.Label(frame_escala2, text="Paths")
label_escala2.pack()

escala2 = Scale(frame_escala2, from_=100, to=0, orient=tk.VERTICAL)
escala2.set(0)
escala2.pack(side = 'left')

# scale = tk.Scrollbar(frame_lateral, orient=tk.VERTICAL)
# scale.pack()



### frame do grafico

In [331]:
frame_grafico = Frame(raiz)

frame_grafico.pack()

### LOOPING DA JANELA PRINCIPAL

In [332]:
raiz.mainloop()

C:\Users\meduarda\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pm4py\utils.py:96: UserWarning: Some rows of the Pandas data frame have been removed because of empty case IDs, activity labels, or timestamps to ensure the correct functioning of PM4Py's algorithms.
  warnings.warn("Some rows of the Pandas data frame have been removed because of empty case IDs, activity labels, or timestamps to ensure the correct functioning of PM4Py's algorithms.")


Imagem do DFG salva como 'dfg_grafico.png'
